# PRE PROCESAMIENTO

### 01 Instalación de paquetes a usar

In [1]:
!pip install xgboost==1.2.0
!pip install numpy
!pip install pandas
!pip install imblearn
!pip install sklearn
!pip install h2o

### 02 Instalación y actualización de Java (para usar H2O)

In [ ]:
!apt-get install openjdk-8-jdk --yes
!apt-get update

### 03 Importación de paquetes a usar e iniciación de H2O

In [ ]:
import numpy as np
import pandas as pd

### 04 Procesamiento de datasets

In [1]:
import pandas as pd
path = ""

# Dataset: 01_suscriptora_202201_202204.csv
df_1 = pd.read_csv("../input/data-final/01_suscriptora_202201_202204.csv")
pd.to_datetime(df_1['FECINGRESOCLIENTE'], format='%Y-%m-%d %H:%M:%S', errors = 'coerce')[23710] is pd.NaT
df_1['TIPO_ADQ']=df_1['TIPO_ADQ'].map({'tipo1':1, 'tipo2':2})
df_1['FECINGRESOCLIENTE'] = pd.to_datetime(df_1['FECINGRESOCLIENTE'], format='%Y-%m-%d %H:%M:%S', errors = 'coerce')
df_1['FECACTIVACIONCONTRATO'] = pd.to_datetime(df_1['FECACTIVACIONCONTRATO'], format='%Y-%m-%d %H:%M:%S', errors = 'coerce')

# Dataset: 02_adenda_202201_202204.csv
df_2 = pd.read_csv("../input/data-final/02_adenda_202201_202204.csv")

# Dataset: 03_perfil_digital_202201_202204.csv
df_3 = pd.read_csv("../input/data-final/03_perfil_digital_202201_202204.csv")
def format_group(x):
    if x != None:
        return int(x[6:])
    return None
df_3['GRUPO'].str.split('|', expand=True).applymap(format_group).max()
for i in range(1,12):
     df_3.loc[0, f'GRUPO_{i}'] = False
        
for index, value in df_3['GRUPO'].items():
    group_list = [int(x[6:]) for x in value.split('|')]
    for group in group_list:
        df_3.loc[index, f'GRUPO_{group}'] = True;

df_3 = df_3.fillna(0)

for i in range (1,12):
    df_3[f'GRUPO_{i}'].replace({False:0}, inplace=True)

for i in range (1,12):
    df_3[f'GRUPO_{i}'].replace({True:1}, inplace=True)
    
df_3.drop(columns=['GRUPO'], inplace=True)

df_3['SCORECAT']=df_3['SCORECAT'].map({'bajo':1, 'medio':2, 'alto':3, 'muy alto':4})


# Dataset: 03_perfil_digital_202201_202204.csv

df_4 = pd.read_csv("../input/data-final/04_roaming_202201_202204.csv")
df_4 = df_4.drop("FECHATRAFICO", axis=1)
df_4[df_4['TIPOSERVICIO']=='TIPO2']['MINUTOS'].value_counts(normalize=True)

filt = (df_4['TIPOSERVICIO'] == 'TIPO2') & (df_4['MINUTOS'] == 0)
df_4.drop(index=df_4[filt].index, inplace=True)

filt = (df_4['TIPOSERVICIO'] == 'TIPO1') & (df_4['GIGAS'] == 0)
df_4.drop(index=df_4[filt].index, inplace=True)

filt = (df_4['TIPOSERVICIO'] == 'TIPO3') & (df_4['MENSAJES'] == 0)
df_4.drop(index=df_4[filt].index, inplace=True)

for i in ["GIGAS", "MINUTOS", "MENSAJES"]:
    b = df_4.groupby(["PERIODO", "nro_telefono_hash"]).agg({i: "sum"})
    df_4 = df_4.drop(i, axis=1).merge(b, how='left', left_on=[
        "PERIODO", "nro_telefono_hash"], right_on=["PERIODO", "nro_telefono_hash"]).drop_duplicates()

df_4.drop(columns=['TIPOSERVICIO'],inplace=True)

# Dataset: 05_terminales_202201_202204.csv

df_5 = pd.read_csv("../input/data-final/05_terminales_202201_202204.csv")


# Dataset: 06_trafico_202201_202204.csv

df_6 = pd.read_csv("../input/data-final/06_trafico_202201_202204.csv")

# Dataset: 07_convergente_202201_202204.csv

df_7 = pd.read_csv("../input/data-final/07_convergente_202201_202204.csv")


# Dataset: 08_target_202201_202203.csv

df_8 = pd.read_csv("../input/data-final/08_target_202201_202203.csv")


### 05 Merge de datasets

In [ ]:
# Merge llaves: NUMPERIODO y nro_telefono_hash
merge_12 = df_1.merge(df_2,how="left",left_on=['NUMPERIODO','nro_telefono_hash'],right_on=['NUMPERIODO','nro_telefono_hash'])
merge_123 = merge_12.merge(df_3,how="left",left_on=['NUMPERIODO','nro_telefono_hash'],right_on=['PERIODO','nro_telefono_hash'])
merge_1234 = merge_123.merge(df_4,how="left",left_on=['NUMPERIODO','nro_telefono_hash'],right_on=['PERIODO','nro_telefono_hash'])
merge_12346 = merge_1234.merge(df_6,how="left",left_on=['NUMPERIODO','nro_telefono_hash'],right_on=['NUMPERIODO','nro_telefono_hash'])
merge_123468 = merge_12346.merge(df_8,how="left",left_on=['NUMPERIODO','nro_telefono_hash'],right_on=['PERIODO','nro_telefono_hash'])

# Merge llaves: NUMPERIODO y nro_telefono_hash
merge_1234568 = merge_123468.merge(df5, how='right', left_on=["PERIODO", "nro_telefono_hash"], right_on=["PERIODO", "nro_telefono_hash"]).drop_duplicates()
merge_12345678 = merge_1234568.merge(df7, how='left', left_on=["NUMPERIODO", "nro_documento_hash"], right_on=["PERIODO", "nro_documento_hash"]).drop_duplicates()


In [ ]:
merge_12345678.to_csv("dataprefinal.csv")